# from-ipfs Demo: Load ML Models from IPFS

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alexbakers/from_ipfs/blob/main/examples/from_ipfs_colab_demo.ipynb)

This notebook demonstrates how to use `from-ipfs` to load models directly from IPFS into:
1. 🤗 Hugging Face Transformers (Sentiment Analysis model)
2. 🦙 llama.cpp (GPT-2 GGUF model)

No setup required - just run each cell in order!

## 1. Installation

First, let's install the required packages:

In [ ]:
!pip install from-ipfs[all] torch --quiet

## 2. Sentiment Analysis with Transformers

Let's use a GPT-2 model fine-tuned for tweet sentiment analysis. The model is loaded directly from IPFS!

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load model from IPFS
model_uri = "ipfs://bafybeichqdarufyutqc7yd43k77fkxbmeuhhetbihd3g32ghcqvijp6fxi"

print("Loading model from IPFS...")
tokenizer = AutoTokenizer.from_pretrained(model_uri)
model = AutoModelForSequenceClassification.from_pretrained(model_uri)
print("Model loaded successfully!\n")

def analyze_sentiment(text):
    # Prepare input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Get model prediction
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Convert to sentiment
    sentiment = "positive" if probabilities[0][1] > 0.5 else "negative"
    confidence = float(probabilities[0][1] if sentiment == "positive" else probabilities[0][0])

    return sentiment, confidence

# Test with example tweets
example_tweets = [
    "I absolutely love this new update! The features are amazing! 😊",
    "This is the worst service I've ever experienced. Very disappointed. 😠",
    "The weather is nice today, perfect for a walk in the park!",
    "This product is not worth the money, don't buy it 👎"
]

for tweet in example_tweets:
    sentiment, confidence = analyze_sentiment(tweet)
    print(f"Tweet: {tweet}")
    print(f"Sentiment: {sentiment} (confidence: {confidence:.2%})\n")

## 3. Text Generation with llama.cpp

Now let's try text generation using a quantized GPT-2 model in GGUF format, loaded directly from IPFS!

In [ ]:
from llama_cpp import Llama

# Load model from IPFS
model_uri = "ipfs://bafybeibawdd2fzph73hh3z2ip2js4wv3ct3kcc2lkyzet4bc75k5bg5xby/openai-community/gpt2.Q5_K_M.gguf"

print("Loading model from IPFS...")
llm = Llama(model_path=model_uri, n_ctx=512)
print("Model loaded successfully!\n")

# Test with different prompts
prompts = [
    "Once upon a time in a magical forest,",
    "The recipe for a perfect chocolate cake includes:",
    "The future of artificial intelligence will"
]

for prompt in prompts:
    print(f"Prompt: {prompt}")

    output = llm(prompt,
                 max_tokens=64,
                 temperature=0.7,
                 top_p=0.95,
                 repeat_penalty=1.1,
                 stop=["\n\n"])

    generated_text = output['choices'][0]['text']
    print(f"Generated: {prompt}{generated_text}\n")

## 4. Cache Information

Let's see what models are cached locally:

In [ ]:
from from_ipfs.utils import list_cached_models, show_config

# Show configuration
print("Current configuration:")
show_config()
print("\nCached models:")
for model in list_cached_models():
    print(f"- {model}")

## That's it!

You've successfully:
1. ✅ Loaded a sentiment analysis model from IPFS
2. ✅ Analyzed tweet sentiments
3. ✅ Loaded a GGUF model from IPFS
4. ✅ Generated text with GPT-2

Want to learn more? Check out:
- 📚 [from-ipfs Documentation](https://github.com/alexbakers/from_ipfs#readme)
- 🤗 [Original Sentiment Model](https://huggingface.co/riturajpandey739/gpt2-sentiment-analysis-tweets)
- 🦙 [Original GGUF Model](https://huggingface.co/PrunaAI/openai-community-gpt2-GGUF-smashed)